In [1]:
import numpy as np,os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, power_transform, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, GridSearchCV, cross_val_predict
from sklearn.utils import shuffle
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, auc, accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, TomekLinks
from imblearn.ensemble import RUSBoostClassifier, BalancedRandomForestClassifier, BalancedBaggingClassifier
from imblearn.pipeline import make_pipeline
import seaborn as sns,matplotlib.pyplot as plt

from library.utils import evaluate, read_data

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-pack

In [2]:
DATASETS = ['groovy-1_5_7.csv','jruby-1.4.0.csv','lucene-2.9.0.csv','jruby-1.7.0.preview1.csv','groovy-1_6_BETA_1.csv',
        'derby-10.2.1.6.csv','wicket-1.5.3.csv','camel-2.9.0.csv','camel-1.4.0.csv','activemq-5.8.0.csv']
DATASETS = [f for f in os.listdir("JIRA/") if 'csv' in f]
len(DATASETS)

32

In [3]:
for d in DATASETS:
    read_data(d,True)

activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)
groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)
activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)
wicket-1.3.0-incubating-beta-1.csv noise:0.164, imb:4.806,288,1384, Shape:(1672, 65)
jruby-1.1.csv noise:0.175, imb:3.540,161,570, Shape:(731, 65)
jruby-1.4.0.csv noise:0.190, imb:3.890,200,778, Shape:(978, 65)
lucene-2.3.0.csv noise:0.204, imb:4.031,160,645, Shape:(805, 65)
hbase-0.95.2.csv noise:0.260, imb:15.088,114,1720, Shape:(1834, 65)
lucene-3.0.0.csv noise:0.185, imb:6.037,190,1147, Shape:(1337, 65)
camel-2.9.0.csv noise:0.044, imb:34.600,200,6920, Shape:(7120, 65)
wicket-1.5.3.csv noise:0.064, imb:26.720,93,2485, Shape:(2578, 65)
lucene-3.1.csv noise:0.120, imb:7.477,331,2475, Shape:(2806, 65)
groovy-1_6_BETA_2.csv noise:0.096, imb:7.583,103,781, Shape:(884, 65)
activemq-5.2.0.csv noise:0.113, imb:12.247,154,1886, Shape:(2040, 65)
groovy-1_5_7.csv noise:0.085, imb:8.463,

In [5]:
imbs = {
    'smote': SMOTE(k_neighbors=5),
    'rus': RandomUnderSampler('not minority'),
    'wilson':EditedNearestNeighbours(n_neighbors=5),  #Default was 3
    'tomek': TomekLinks(),
}
clfs = {
    'dt': DecisionTreeClassifier(max_depth=20),
    'lr': LogisticRegression(solver='lbfgs',max_iter=1000),
    'nb': GaussianNB(),
    'knn': KNeighborsClassifier(n_neighbors=5),
    'rf': RandomForestClassifier(n_estimators=50),
}
ensembles = {
    'rboost_DT': RUSBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10),algorithm='SAMME',n_estimators=10),
    'rboost_NB': RUSBoostClassifier(base_estimator=GaussianNB(),algorithm='SAMME',n_estimators=10),
    'bbag_DT': BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=20,max_features='sqrt')),
    'bbag_NB': BalancedBaggingClassifier(base_estimator=GaussianNB()),
}
simples = {
    'LR': LogisticRegression(solver='lbfgs',max_iter=1000),
    'RF': RandomForestClassifier(n_estimators=50)
}

In [6]:
models = {}
for im,samp in imbs.items():
    for c,clf in clfs.items():
        models[(im,c)] = make_pipeline(samp,clf)

for m,ens in ensembles.items():
    models[('ens',m)] = ens
    
for m,clf in simples.items():
    models[('sim',m)] = clf    
    
models.keys(),len(models)

(dict_keys([('smote', 'dt'), ('smote', 'lr'), ('smote', 'nb'), ('smote', 'knn'), ('smote', 'rf'), ('rus', 'dt'), ('rus', 'lr'), ('rus', 'nb'), ('rus', 'knn'), ('rus', 'rf'), ('wilson', 'dt'), ('wilson', 'lr'), ('wilson', 'nb'), ('wilson', 'knn'), ('wilson', 'rf'), ('tomek', 'dt'), ('tomek', 'lr'), ('tomek', 'nb'), ('tomek', 'knn'), ('tomek', 'rf'), ('ens', 'rboost_DT'), ('ens', 'rboost_NB'), ('ens', 'bbag_DT'), ('ens', 'bbag_NB'), ('sim', 'LR'), ('sim', 'RF')]),
 26)

In [22]:
models = {
    str(c):LogisticRegression(C=c) for c in  [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

In [23]:
cv = RepeatedStratifiedKFold(n_splits=10,n_repeats=3,random_state=42)
def pr_rec_score(y,yp):
    prec, rec, _ = precision_recall_curve(y,yp)
    return auc(rec,prec)
scorers = [matthews_corrcoef,pr_rec_score]

In [18]:
cols = pd.MultiIndex.from_product([imbs.keys(),clfs.keys(),[f.__name__ for f in scorers]],names=['imb','clf','metric'])
df = pd.DataFrame(index=DATASETS,columns=cols)
#df = pd.read_csv("Imb X Clf-Noisy.csv",header=[0,1,2],index_col=0)

In [19]:
for k in models:
    print(k)
    for d in DATASETS:
        try:
            if df.loc[d,(k[0],k[1],slice(None))].isna().sum()==0: 
                print("Skipping ",d,k[0],k[1])
                continue
        except:
            pass
        X,y_noisy,y_real = read_data(d,stats=True)
        r = evaluate(models[k],X,y_noisy,y_real,cv,scorers)
        for f in r:
            df.loc[d,(k[0],k[1],f)] = r[f].mean()

0.001
activemq-5.8.0.csv noise:0.058, imb:15.847,203,3217, Shape:(3420, 65)


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

groovy-1_6_BETA_1.csv noise:0.128, imb:6.017,117,704, Shape:(821, 65)


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

activemq-5.3.0.csv noise:0.094, imb:15.669,142,2225, Shape:(2367, 65)


...

hive-0.9.0.csv noise:0.179, imb:25.717,53,1363, Shape:(1416, 65)


/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/metrics/_classification.py:896: RuntimeWarning: invalid value encountered in double_scalars
  mcc = 

0.01
Skipping  activemq-5.8.0.csv 0 .
Skipping  groovy-1_6_BETA_1.csv 0 .
Skipping  activemq-5.3.0.csv 0 .
Skipping  wicket-1.3.0-incubating-beta-1.csv 0 .
Skipping  jruby-1.1.csv 0 .
Skipping  jruby-1.4.0.csv 0 .
Skipping  lucene-2.3.0.csv 0 .
Skipping  hbase-0.95.2.csv 0 .
Skipping  lucene-3.0.0.csv 0 .
Skipping  camel-2.9.0.csv 0 .
Skipping  wicket-1.5.3.csv 0 .
Skipping  lucene-3.1.csv 0 .
Skipping  groovy-1_6_BETA_2.csv 0 .
Skipping  activemq-5.2.0.csv 0 .
Skipping  groovy-1_5_7.csv 0 .
Skipping  hbase-0.95.0.csv 0 .
Skipping  camel-2.11.0.csv 0 .
Skipping  jruby-1.7.0.preview1.csv 0 .
Skipping  hive-0.10.0.csv 0 .
Skipping  camel-2.10.0.csv 0 .
Skipping  derby-10.2.1.6.csv 0 .
Skipping  jruby-1.5.0.csv 0 .
Skipping  derby-10.3.1.4.csv 0 .
Skipping  lucene-2.9.0.csv 0 .
Skipping  hbase-0.94.0.csv 0 .
Skipping  camel-1.4.0.csv 0 .
Skipping  activemq-5.1.0.csv 0 .
Skipping  activemq-5.0.0.csv 0 .
Skipping  derby-10.5.1.1.csv 0 .
Skipping  wicket-1.3.0-beta2.csv 0 .
Skipping  hive-0.

/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/shihab/anaconda3/envs/ana/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/mo

IndexError: string index out of range

In [20]:
df.to_csv("LR-C vs Perf.csv")

In [21]:
df.shape

(32, 42)

## Effect of Noise on Performance

In [ ]:
noise = pd.read_csv("Imb X Clf- Noise.csv",header=[0,1,2],index_col=0)
clean = pd.read_csv("Imb X Clf- Clean.csv",header=[0,1,2],index_col=0)
noise.shape,clean.shape

In [ ]:
wilcoxon(noise.values.reshape(-1),clean.values.reshape(-1))

In [ ]:
noise.values.mean(),clean.values.mean()

## Statistical Analysis

In [ ]:
from scipy.stats import wilcoxon, friedmanchisquare
import scikit_posthocs as sp

In [ ]:
boo = df.rename_axis('Datasets').reset_index()

In [ ]:
res = pd.melt(boo,id_vars=['Datasets'])
res.columns

In [ ]:
res

### Compare DT & RF & NB

In [ ]:
mathew =  res[res['metric']=='matthews_corrcoef'].drop(columns=['metric'])

In [ ]:
mathew = mathew[mathew['clf'].isin(['dt','rf','nb'])]
mathew.shape

In [ ]:
tmp = mathew.set_index(['Datasets','imb'])
tmp

In [ ]:
hey = tmp.pivot(columns='clf')
hey

In [ ]:
dog = hey.reset_index(drop=True)
dog.columns = dog.columns.droplevel(0)
dog

In [ ]:
friedmanchisquare(*(dog[c] for c in dog.columns))

In [ ]:
dog.mean()f

In [ ]:
pvals = pd.DataFrame(columns=dog.columns,index=dog.columns,dtype='float')
for c in dog.columns:
    for d in dog.columns:
        if c==d: continue
        pvals.loc[c,d] = float(wilcoxon(dog[c],dog[d]).pvalue)
pvals

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
ps = pvals.values.reshape(-1)
idx = np.isnan(ps)
idx

In [ ]:
ps[~idx]

In [ ]:
multipletests(ps[~idx],method='fdr_by')